### Расчетная работа по теории вероятностей № 1

In [1]:
with open('input.txt', encoding='utf8') as f:
    content = f.readlines()
    print(content)

['Число букв: 222 Разрядность кода: 7 бит Шум: 0.168 Число посылок: 18\n', 'Зашумленная посылка №  1: 0001000 1001111 0010110 0001110 1000110 1001001 1000011 0110100 0111000 0101101 1010110 0000100 0110011 0001101 0100100 0111100 1011101 1011010 0110100 0110011 1000110 0101111 0111011 0011010 0110011 0110010 1001111 1110010 0000010 0000011 0000101 1000010 0000001 0010000 1010110 1011101 1010110 0101101 0110010 1100011 0011000 0011000 1110010 0111011 0110110 1010110 1101101 0110110 0001011 0011000 0111011 1000111 0111011 0101010 0110111 0101111 1000011 0111110 1010010 0110011 0101011 1001011 1110100 1111110 1010110 0111011 0101010 1010110 0101110 0100000 0011000 0111010 0110100 0101100 1010100 0101101 0110000 0101101 0101010 1000011 0111110 0101001 0111010 0111101 0110101 0110100 0110001 1000010 0111111 0010010 0011000 0110100 0111110 0100100 0111110 0110100 0111101 0100011 1010110 1011100 0110100 0111100 0110000 0110111 0100101 0101011 0010111 0111100 0000011 1000011 0100001 0011101 01

In [2]:
dic = {}
with open('alphabet.txt', encoding='utf8') as f:
    content = f.readlines()
    content = [x.rstrip().split('\t') for x in content] 
    for pair in content:
        dic[pair[0]] = pair[1]
    print(len(dic))
    print(dic)
    print(dic['?'])

87
{'щ': '1000101', 'А': '0001010', '8': '0001000', 'ж': '0110010', 'Ё': '0010000', '1': '0000001', 'Д': '0001110', 'ё': '0110001', 'С': '0011100', 'Я': '0101010', 'Ж': '0010001', ':': '1001111', 'ц': '1000010', 'К': '0010101', 'Г': '0001101', '4': '0000100', 'з': '0110011', 'Ч': '0100010', 'т': '0111110', 'ъ': '1001000', 'Л': '0010110', 'Б': '0001011', 'О': '0011001', ')': '1010101', '0': '0000000', 'Ю': '0101001', 'Ь': '0100101', '_': '1010010', ',': '1001101', 'В': '0001100', '-': '1010001', '7': '0000111', 'ч': '1000011', 'М': '0010111', 'е': '0110000', 'Е': '0001111', 'й': '0110101', '?': '1010000', 'а': '0101011', '3': '0000011', 'З': '0010010', '.': '1001100', 'о': '0111010', 'Н': '0011000', 'л': '0110111', 'Ъ': '0100111', 'б': '0101100', 'Ш': '0100011', 'Ф': '0011111', 'у': '0111111', 'ю': '1001010', 'У': '0011110', 'ы': '1000111', 'Х': '0100000', 'н': '0111001', 'Ц': '0100001', '№': '1010011', 'Р': '0011011', ' ': '1010110', 'Й': '0010100', 'в': '0101101', 'с': '0111101', '5':